### Import Dependencies

In [53]:
from qdrant_client import QdrantClient
from qdrant_client.models import Prefetch, Filter, FieldCondition, MatchText, FusionQuery, Document, VectorParams, Distance, PayloadSchemaType, PointStruct, MatchAny

import pandas as pd
import numpy as np
import openai
import json
import tiktoken

#### Retrieve all item IDs from Amazon Items Qdrant Collection

In [2]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [3]:
dummy_vector = np.zeros(1536).tolist()

In [4]:
payload = qdrant_client.query_points(
    collection_name="Amazon-items-collection-01-hybrid-search-v2",
    query=dummy_vector,
    using="text-embedding-3-small",
    limit=1000,
    with_payload=["parent_asin"],
    with_vectors=False
)

In [5]:
payload.points

[ScoredPoint(id=154, version=3, score=0.0, payload={'parent_asin': 'B0BCJ2X4GW'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=464, version=3, score=0.0, payload={'parent_asin': 'B0C4352BSQ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=268, version=3, score=0.0, payload={'parent_asin': 'B0BM75JNKW'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=298, version=3, score=0.0, payload={'parent_asin': 'B0BXNNRBS2'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=13, version=3, score=0.0, payload={'parent_asin': 'B0BJRY9GHK'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=243, version=3, score=0.0, payload={'parent_asin': 'B0C7C6KN4R'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=303, version=3, score=0.0, payload={'parent_asin': 'B0CKN6TT18'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=396, version=3, score=0.0, payload={'parent_asin': 'B0C2TK8NB4'}, vector=No

In [6]:
len(payload.points)

1000

In [7]:
parent_asin_list = [item.payload["parent_asin"] for item in payload.points]

In [8]:
parent_asin_list

['B0BCJ2X4GW',
 'B0C4352BSQ',
 'B0BM75JNKW',
 'B0BXNNRBS2',
 'B0BJRY9GHK',
 'B0C7C6KN4R',
 'B0CKN6TT18',
 'B0C2TK8NB4',
 'B0BX946QHN',
 'B09PVC7RYZ',
 'B0B67ZFRPC',
 'B09QPQRZCZ',
 'B0BWF8TTXP',
 'B09T5KL86Z',
 'B0B4DGNN94',
 'B0BKZ7D2F3',
 'B0C4J5GH3W',
 'B0B7RZY38W',
 'B097CYFH1M',
 'B0BPLQ5Z2F',
 'B0C37TQ5B3',
 'B09WDJST3X',
 'B0B9MXDXCT',
 'B0BRT3PCYJ',
 'B0BKFKSRHR',
 'B0BC3YWYB9',
 'B09KMKKWVJ',
 'B0C7MFC95N',
 'B0BKZ2SX5H',
 'B0B5K177FX',
 'B09GLCSV75',
 'B09VFCM2TF',
 'B096BND451',
 'B0BPSS74BX',
 'B0CFL6G1HF',
 'B0BFCDCPZC',
 'B0B9G3HBCD',
 'B09SZ5XMXD',
 'B0BLP6CHY3',
 'B09X73T9DQ',
 'B0BW9RHFK4',
 'B0B9SDZSGZ',
 'B09YXPJHBY',
 'B0BZP8K7Y7',
 'B09BYLJYTH',
 'B09PQXS35S',
 'B0BLTVDZ8F',
 'B09XQXGXZR',
 'B0C5JDHYQR',
 'B0C1CRNRPT',
 'B0CGMHWFSN',
 'B0BC1D91N1',
 'B0C5H7CYHG',
 'B0CB1NJGH9',
 'B0BVFMC7PV',
 'B09T35VLX2',
 'B0C22QDH2K',
 'B0BD2HJQVV',
 'B09LXQ7G31',
 'B08XMNYYWR',
 'B0BWQKBX1N',
 'B0BN54T6XD',
 'B0BLMTQZ4Q',
 'B09TF16ZQB',
 'B0BR8TSXTT',
 'B0C77PFNK2',
 'B0C1JZD8

#### LoadAmazon Reviews Dataset

In [11]:
df_reviews = pd.read_json("../../data/Electronics_2022_2023_with_category_rating_100_sample_1000.jsonl", lines=True)

In [12]:
df_reviews.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4,Nixplay 10.1 touch screen digital picture frame,I purchased this digital frame on a treasure t...,[],B096DQF21Z,B0BNXXNBB4,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-07-29 06:52:30.702,19,True
1,5,charging cable USB C,These are great. They don't coil really great...,[],B096XFSFW3,B09DG877JS,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-06-21 21:52:11.907,0,True
2,3,Perfect for protecting remote,Nice covers,[],B07CGBB685,B08BWN7F7Q,AGA77XVPYPRDNC3AJAIC557RGSLA,2021-03-13 07:42:11.049,0,True
3,5,Totally awesome,Will buy em again,[],B09QHT56V5,B0CB11DLHG,AF37HETXCNPLIXJRMMYWVDLI4QBQ,2022-03-25 14:11:02.409,0,True
4,5,I would buy it again.,Great product. Worked well for what we needed ...,[],B07T55DL33,B0B2JWCMCY,AF5KFHNT3TQJ2GNSE3FCDFQOBICA,2019-12-09 22:35:00.531,0,True


In [13]:
len(df_reviews)

98302

In [14]:
df_reviews_sample = df_reviews[df_reviews["parent_asin"].isin(parent_asin_list)]

In [15]:
df_reviews_sample.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4,Nixplay 10.1 touch screen digital picture frame,I purchased this digital frame on a treasure t...,[],B096DQF21Z,B0BNXXNBB4,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-07-29 06:52:30.702,19,True
1,5,charging cable USB C,These are great. They don't coil really great...,[],B096XFSFW3,B09DG877JS,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-06-21 21:52:11.907,0,True
2,3,Perfect for protecting remote,Nice covers,[],B07CGBB685,B08BWN7F7Q,AGA77XVPYPRDNC3AJAIC557RGSLA,2021-03-13 07:42:11.049,0,True
3,5,Totally awesome,Will buy em again,[],B09QHT56V5,B0CB11DLHG,AF37HETXCNPLIXJRMMYWVDLI4QBQ,2022-03-25 14:11:02.409,0,True
4,5,I would buy it again.,Great product. Worked well for what we needed ...,[],B07T55DL33,B0B2JWCMCY,AF5KFHNT3TQJ2GNSE3FCDFQOBICA,2019-12-09 22:35:00.531,0,True


In [16]:
len(df_reviews_sample)

98302

#### Define functions to preprocess reviews data

In [17]:
def preprocess_reviews_data(row):
    return f"{row['title']} {row['text']}"

In [19]:
encoding  = tiktoken.encoding_for_model("text-embedding-3-small")

In [20]:
encoding.encode("Can I get some earphones")

[6854, 358, 636, 1063, 2487, 17144]

In [21]:
def token_count(row, model="text-embedding-3-small"):

    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(row["preprocess_data"]))

In [22]:
df_reviews_sample["preprocess_data"] = df_reviews_sample.apply(preprocess_reviews_data, axis=1)

In [23]:
df_reviews_sample.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,preprocess_data
0,4,Nixplay 10.1 touch screen digital picture frame,I purchased this digital frame on a treasure t...,[],B096DQF21Z,B0BNXXNBB4,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-07-29 06:52:30.702,19,True,Nixplay 10.1 touch screen digital picture fram...
1,5,charging cable USB C,These are great. They don't coil really great...,[],B096XFSFW3,B09DG877JS,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-06-21 21:52:11.907,0,True,charging cable USB C These are great. They do...
2,3,Perfect for protecting remote,Nice covers,[],B07CGBB685,B08BWN7F7Q,AGA77XVPYPRDNC3AJAIC557RGSLA,2021-03-13 07:42:11.049,0,True,Perfect for protecting remote Nice covers
3,5,Totally awesome,Will buy em again,[],B09QHT56V5,B0CB11DLHG,AF37HETXCNPLIXJRMMYWVDLI4QBQ,2022-03-25 14:11:02.409,0,True,Totally awesome Will buy em again
4,5,I would buy it again.,Great product. Worked well for what we needed ...,[],B07T55DL33,B0B2JWCMCY,AF5KFHNT3TQJ2GNSE3FCDFQOBICA,2019-12-09 22:35:00.531,0,True,I would buy it again. Great product. Worked we...


In [24]:
df_reviews_sample["preprocess_data_token_count"] = df_reviews_sample.apply(token_count, axis=1)

In [25]:
df_reviews_sample.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,preprocess_data,preprocess_data_token_count
0,4,Nixplay 10.1 touch screen digital picture frame,I purchased this digital frame on a treasure t...,[],B096DQF21Z,B0BNXXNBB4,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-07-29 06:52:30.702,19,True,Nixplay 10.1 touch screen digital picture fram...,255
1,5,charging cable USB C,These are great. They don't coil really great...,[],B096XFSFW3,B09DG877JS,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-06-21 21:52:11.907,0,True,charging cable USB C These are great. They do...,32
2,3,Perfect for protecting remote,Nice covers,[],B07CGBB685,B08BWN7F7Q,AGA77XVPYPRDNC3AJAIC557RGSLA,2021-03-13 07:42:11.049,0,True,Perfect for protecting remote Nice covers,6
3,5,Totally awesome,Will buy em again,[],B09QHT56V5,B0CB11DLHG,AF37HETXCNPLIXJRMMYWVDLI4QBQ,2022-03-25 14:11:02.409,0,True,Totally awesome Will buy em again,7
4,5,I would buy it again.,Great product. Worked well for what we needed ...,[],B07T55DL33,B0B2JWCMCY,AF5KFHNT3TQJ2GNSE3FCDFQOBICA,2019-12-09 22:35:00.531,0,True,I would buy it again. Great product. Worked we...,24


In [26]:
len(df_reviews_sample)

98302

In [27]:
df_reviews_sample = df_reviews_sample[df_reviews_sample["preprocess_data_token_count"] < 8192]

In [28]:
len(df_reviews_sample)

98302

In [29]:
total_tokens = df_reviews_sample["preprocess_data_token_count"].sum()

In [30]:
total_tokens

np.int64(5734425)

#### Create a new Qdrant Collection for Reviews

In [ ]:
qdrant_client.create_collection(
    collection_name="Amazon-reviews-collection-01-reviews",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

True

In [38]:
qdrant_client.create_payload_index(
    collection_name="Amazon-reviews-collection-01-reviews",
    field_name="parent_asin",
    field_schema=PayloadSchemaType.KEYWORD
)


UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

#### Embedding Functions

In [57]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        model=model,
        input=text
    )
    return response.data[0].embedding

In [46]:
def get_embeddings_batch(text_list, model="text-embedding-3-small", batch_size=100):

    if len(text_list) <= batch_size:
        response = openai.embeddings.create(input=text_list, model=model)
        return [embedding.embedding for embedding in response.data]

    all_embeddings = []
    counter = 1
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i+batch_size]
        response = openai.embeddings.create(input=batch, model=model)
        all_embeddings.extend([embedding.embedding for embedding in response.data])
        print(f"Processed batch {counter * batch_size} of {len(text_list)}")
        counter += 1
        
    return all_embeddings
    

#### Embed the text and add additional fields to the payload of each vector for reviews

In [41]:
data_to_embed_reviews = df_reviews_sample[["preprocess_data", "parent_asin"]].to_dict(orient="records")

In [42]:
data_to_embed_reviews

[{'preprocess_data': "Nixplay 10.1 touch screen digital picture frame I purchased this digital frame on a treasure truck deal - it  works.  You will have to load the app and follow directions to bluetooth transfer pics you choose from your gallery to the frame.  I purchased an additional memory chip - but it turns out there is no place to install the chip so don't waste your time there.  You can set a regular time to 'run' the frame or manually turn it off and/or on.  You can 'shuffle' OR play all your uploaded pics.  You do not get to choose the 'fade' that happens between pics, but you do get to choose how long each pic stays on the frame before moving to the next pic. Picture quality is pretty good - about as good as the pics you download on to it.  It's close to an 8x10 frame.  Not sure why they call it a 'touch screen' cause I can't figure out what you can do by touching the screen - but ok. It comes with charging cord, regular plug in - there appears to be a mini USB port, but I 

In [43]:
text_to_embed_reviews = [data["preprocess_data"] for data in data_to_embed_reviews]

In [44]:
text_to_embed_reviews

["Nixplay 10.1 touch screen digital picture frame I purchased this digital frame on a treasure truck deal - it  works.  You will have to load the app and follow directions to bluetooth transfer pics you choose from your gallery to the frame.  I purchased an additional memory chip - but it turns out there is no place to install the chip so don't waste your time there.  You can set a regular time to 'run' the frame or manually turn it off and/or on.  You can 'shuffle' OR play all your uploaded pics.  You do not get to choose the 'fade' that happens between pics, but you do get to choose how long each pic stays on the frame before moving to the next pic. Picture quality is pretty good - about as good as the pics you download on to it.  It's close to an 8x10 frame.  Not sure why they call it a 'touch screen' cause I can't figure out what you can do by touching the screen - but ok. It comes with charging cord, regular plug in - there appears to be a mini USB port, but I haven't tried it.  H

In [47]:
embeddings_reviews = get_embeddings_batch(text_to_embed_reviews, batch_size=500)

Processed batch 500 of 98302
Processed batch 1000 of 98302
Processed batch 1500 of 98302
Processed batch 2000 of 98302
Processed batch 2500 of 98302
Processed batch 3000 of 98302
Processed batch 3500 of 98302
Processed batch 4000 of 98302
Processed batch 4500 of 98302
Processed batch 5000 of 98302
Processed batch 5500 of 98302
Processed batch 6000 of 98302
Processed batch 6500 of 98302
Processed batch 7000 of 98302
Processed batch 7500 of 98302
Processed batch 8000 of 98302
Processed batch 8500 of 98302
Processed batch 9000 of 98302
Processed batch 9500 of 98302
Processed batch 10000 of 98302
Processed batch 10500 of 98302
Processed batch 11000 of 98302
Processed batch 11500 of 98302
Processed batch 12000 of 98302
Processed batch 12500 of 98302
Processed batch 13000 of 98302
Processed batch 13500 of 98302
Processed batch 14000 of 98302
Processed batch 14500 of 98302
Processed batch 15000 of 98302
Processed batch 15500 of 98302
Processed batch 16000 of 98302
Processed batch 16500 of 983

In [49]:
len(embeddings_reviews)

98302

In [51]:
pointstructs = []
i = 1
for embedding, data in zip(embeddings_reviews, data_to_embed_reviews):
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embedding,
            payload={
                "text": data["preprocess_data"],
                "parent_asin": data["parent_asin"]
            }
        )
    )
    i+=1

In [52]:
batch_size_qdrant = 100
counter = 1
for i in range(0, len(pointstructs), batch_size_qdrant):
    batch = pointstructs[i:i+batch_size_qdrant]
    qdrant_client.upsert(
        collection_name="Amazon-reviews-collection-01-reviews",
        wait=True,
        points=batch
    )
    print(f"Processed batch {counter * batch_size_qdrant} of {len(pointstructs)}")
    counter += 1


Processed batch 100 of 98302
Processed batch 200 of 98302
Processed batch 300 of 98302
Processed batch 400 of 98302
Processed batch 500 of 98302
Processed batch 600 of 98302
Processed batch 700 of 98302
Processed batch 800 of 98302
Processed batch 900 of 98302
Processed batch 1000 of 98302
Processed batch 1100 of 98302
Processed batch 1200 of 98302
Processed batch 1300 of 98302
Processed batch 1400 of 98302
Processed batch 1500 of 98302
Processed batch 1600 of 98302
Processed batch 1700 of 98302
Processed batch 1800 of 98302
Processed batch 1900 of 98302
Processed batch 2000 of 98302
Processed batch 2100 of 98302
Processed batch 2200 of 98302
Processed batch 2300 of 98302
Processed batch 2400 of 98302
Processed batch 2500 of 98302
Processed batch 2600 of 98302
Processed batch 2700 of 98302
Processed batch 2800 of 98302
Processed batch 2900 of 98302
Processed batch 3000 of 98302
Processed batch 3100 of 98302
Processed batch 3200 of 98302
Processed batch 3300 of 98302
Processed batch 340

#### A function to run serach agasint reviews on a prefiltered set of product IDs

In [54]:
def retrieve_prefiltered_reviews_data(query, parent_asins, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-reviews-collection-01-reviews",
        prefetch=Prefetch(
            query=query_embedding,
            filter=Filter(
                must=[
                    FieldCondition(
                        key="parent_asin",
                        match=MatchAny(any=parent_asins)
                    )
                ]
            ),
            limit=20
        ),
        query=FusionQuery(fusion='rrf'),
        limit=k
    )

    return results

In [58]:
reviews = retrieve_prefiltered_reviews_data("bad quality?", ["B0B9JJ1D8Y"], k=5)

In [60]:
reviews.points

[ScoredPoint(id=87918, version=882, score=0.5, payload={'text': 'Dumb You get what you pay for. I should have known better than to be a cheapskate again!', 'parent_asin': 'B0B9JJ1D8Y'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=69142, version=694, score=0.33333334, payload={'text': 'DISPLAY SUCKS The display is blurry even after adjusting every which way including loose. Poor contrast and resolution after adjusting with settings and Intel controls. The flat matt finish makes for an almost blurry image in all modes.<br />  Battery dies overnight even when shut down. Fails to charge and lasts about 1/2 an hour when unplugged from power supply. Worst computer I have ever owned.<br />EDIT: Battery dies within hours of unplugging power supply even though computer is off. Will not even start now until power cord is plugged in for an hour. Can only be used if power cord is attached.', 'parent_asin': 'B0B9JJ1D8Y'}, vector=None, shard_key=None, order_value=None),
 ScoredPo

In [63]:
reviews = retrieve_prefiltered_reviews_data("bad quality?", ["B0B9JJ1D8Y","B09XLN2C6X"], k=20)

In [64]:
reviews.points

[ScoredPoint(id=87750, version=880, score=0.5, payload={'text': 'poor quality was only able to transfer 1 cassette. stopped working after first transfer. returned', 'parent_asin': 'B09XLN2C6X'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=65050, version=653, score=0.33333334, payload={'text': 'Sound and record quality are horrible. Sound and record quality are horrible.', 'parent_asin': 'B09XLN2C6X'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=64218, version=645, score=0.25, payload={'text': 'Poor Quality Device ate my tape on first try. Terrible product and cheaply made.', 'parent_asin': 'B09XLN2C6X'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=35167, version=354, score=0.2, payload={'text': 'Nice idea, very poor quality Bought to easily convert some cassette tapes to digital. Audio quality is very poor - lots of motor hum. Returned it and went a different direction with a higher quality setup<br />(For about the same price

In [65]:
for point in reviews.points:
    print(point.payload["parent_asin"])
    print(point.payload["text"])
    print("-"*100)


B09XLN2C6X
poor quality was only able to transfer 1 cassette. stopped working after first transfer. returned
----------------------------------------------------------------------------------------------------
B09XLN2C6X
Sound and record quality are horrible. Sound and record quality are horrible.
----------------------------------------------------------------------------------------------------
B09XLN2C6X
Poor Quality Device ate my tape on first try. Terrible product and cheaply made.
----------------------------------------------------------------------------------------------------
B09XLN2C6X
Nice idea, very poor quality Bought to easily convert some cassette tapes to digital. Audio quality is very poor - lots of motor hum. Returned it and went a different direction with a higher quality setup<br />(For about the same price)
----------------------------------------------------------------------------------------------------
B09XLN2C6X
Piece of Crap Very Very inferior
--------------